In [56]:
!export LANGCHAIN_TRACING_V2=true
!export LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
!export LANGCHAIN_API_KEY="ls__929418e7969244749567458f226e2223"
!export LANGCHAIN_PROJECT="docreader"

In [33]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [34]:
loader = PyPDFLoader("data/Hallgatoi_Kovetelmenyrendszer_2023._oktober_1..pdf")

In [35]:
data = loader.load()

In [36]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

You have 300 document(s) in your data
There are 3276 characters in your document


In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [38]:
print (f'Now you have {len(texts)} documents')

Now you have 528 documents


In [39]:
from langchain.vectorstores import  Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

In [49]:
with open('settings.json') as settings_file:
    settings = json.load(settings_file)


OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY', settings['OPENAI-API'])
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', settings['PINECONE-API'])
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-west4-gcp')


In [50]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [42]:
import pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV
)
index_name = "projekt"

In [51]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

AuthenticationError: Incorrect API key provided: sk-CZZtQ***************************************d7La. You can find your API key at https://platform.openai.com/account/api-keys.

In [52]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [53]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm,chain_type="stuff")

In [ ]:
query = "Mik a főbb követelmények egy hallagtó számára a dokumentum alapján?"
docs = docsearch.similarity_search(query,include_metadata=True)